# Natural Language Processing using Transformers

... using the [HuggingFace][1] library

[1]: https://huggingface.co/docs/transformers/installation :hugs:

## Setup

In [ ]:
from pathlib import Path
import pandas as pd
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    TrainingArguments,
    Trainer,
    AutoModelForSequenceClassification,
)
import numpy as np

In [ ]:
ROOT_PATH = Path.cwd().parent
DATA_PATH = ROOT_PATH / "data"
DATA_PATH.joinpath("nlp").mkdir(exist_ok=True)

## Load data

In [ ]:
train_df = pd.read_csv(DATA_PATH / "nlp" / "train.csv")

In [ ]:
test_df = pd.read_csv(DATA_PATH / "nlp" / "test.csv")

In [ ]:
train_df.info()

In [ ]:
test_df.info()

In [ ]:
train_df.head()

## Process data

### Create input column

In [ ]:
def make_input_col(df: pd.DataFrame) -> pd.DataFrame:
    df["input"] = (
        "TEXT1: "
        + df["context"]
        + "; TEXT2: "
        + df["target"]
        + "; ANC1: "
        + df["anchor"]
    )
    return df

In [ ]:
train_df = make_input_col(df=train_df)

In [ ]:
test_df = make_input_col(df=test_df)

In [ ]:
train_df["input"].head()

### Tokenize

In [ ]:
train_ds = Dataset.from_pandas(train_df)

In [ ]:
train_ds

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-v3-small")

In [ ]:
tok_train_ds = train_ds.map(lambda x: tokenizer(x["input"]))

In [ ]:
tok_train_ds[0]

In [ ]:
tok_train_ds = tok_train_ds.rename_column(
    original_column_name="score", new_column_name="labels"
)

In [ ]:
tok_train_ds

In [ ]:
tok_test_ds = Dataset.from_pandas(test_df).map(lambda x: tokenizer(x["input"]))

In [ ]:
tok_test_ds

In [ ]:
tok_dds = tok_train_ds.train_test_split(0.25, seed=1337)

In [ ]:
tok_dds

## Train model

In [ ]:
def calc_corr(x, y):
    return np.corrcoef(x, y)[0][1]

In [ ]:
def get_corr_metric(pred):
    return {"pearson": calc_corr(*pred)}

In [ ]:
bs = 128
epochs = 4
lr = 8e-5

In [ ]:
args = TrainingArguments(
    output_dir="outputs",
    learning_rate=lr,
    warmup_ratio=0.1,
    lr_scheduler_type="cosine",
    fp16=True,
    evaluation_strategy="epoch",
    per_device_train_batch_size=bs,
    per_device_eval_batch_size=bs * 2,
    num_train_epochs=epochs,
    weight_decay=-0.01,
    report_to="none",
)

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(
    "microsoft/deberta-v3-small", num_labels=1
)

In [ ]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tok_dds["train"],
    eval_dataset=tok_dds["test"],
    tokenizer=tokenizer,
    compute_metrics=get_corr_metric,
)

In [ ]:
trainer.train()

### Generate predictions

In [ ]:
preds = trainer.predict(test_dataset=tok_test_ds)

In [ ]:
preds.predictions[:10]

## Submit to Kaggle

In [ ]:
sub_df = tok_test_ds.remove_columns(
    column_names=[
        "anchor",
        "target",
        "context",
        "input",
        "input_ids",
        "token_type_ids",
        "attention_mask",
    ]
).to_pandas()

In [ ]:
sub_df["score"] = preds.predictions

In [ ]:
sub_df.loc[sub_df["score"] < 0, "score"] = 0

In [ ]:
sub_df.to_csv(DATA_PATH / "nlp" / "submission.csv")